<img width="50px" align="left" style="margin-right:20px" src="http://data.newprolab.com/public-newprolab-com/npl_logo.png"> <b>New Professions Lab</b> <br /> Специалист по большим данным

# Решение проекта 1

# Спрогнозировать пол и возрастную категорию интернет-пользователей по логу посещения сайтов

<img width="110px" align="left" src="http://data.newprolab.com/public-newprolab-com/project01_img0.png?img">

Одна из задач DMP-системы состоит в том, чтобы по разрозненным даннным, таким, как посещения неким пользователем сайтов, классифицировать его и присвоить ему определённую категорию: пол, возраст, интересы и так далее. В дальнейшем составляется портрет, или профиль, пользователя, на основе которого ему более таргетированно показывается реклама в интернете.

### Задача

Используя доступный набор данных о посещении страниц у одной части пользователей, сделать прогноз относительно **пола и возрастной категории** другой части пользователей. Угадывание (hit) - правильное предсказание и пола, и возрастной категории одновременно.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os, sys
import json
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote

### Обработка данных на вход

Загрузим весь датасет в pandas:

In [2]:
data_dir = 'data'
filename = 'gender_age_dataset.txt'
file_path = '/'.join([data_dir,filename])
df = pd.read_csv(file_path, sep='\t')

И теперь попробуем понять, что у нас есть:

In [3]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


Скомбинируем пол и возраст вединую категорию

In [4]:
df['target'] = df[['gender','age']].apply(lambda x: x['gender']+x['age'], axis=1)
df['target'].unique()

array(['F18-24', 'M25-34', 'F25-34', 'M>=55', 'F45-54', 'F35-44',
       'M35-44', 'F>=55', 'M18-24', 'M45-54', '--'], dtype=object)

Что содержится в `user_json`?

In [5]:
df.iloc[0].user_json

'{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}'

Видим, что это некая сериализованная json-строка, которую можно легко разобрать через модуль `json`:

In [6]:
j = json.loads(df.iloc[0].user_json)
j

{'visits': [{'timestamp': 1419688144068,
   'url': 'http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun'},
  {'timestamp': 1426666298001,
   'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story'},
  {'timestamp': 1426666298000,
   'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html'},
  {'timestamp': 1426661722001,
   'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story'},
  {'timestamp': 1426661722000,
   'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html'}]}

### Очистка данных и feature engineering

Одна из первых вещей, которые можно попробовать — это вытащить домены и использовать их в качестве признаков. Можно воспользоваться функцией:

In [7]:
def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    #if netloc is not None: return str(netloc.encode('utf8')).strip()
    if netloc is not None: return str(netloc.strip())
    return None

In [8]:
def get_domains(j):
    return " ".join([url2domain(d['url']) for d in json.loads(j)['visits'] ])

In [9]:
df['user_json'].head().apply(get_domains, 0)

0    zebra-zoya.ru news.yandex.ru sotovik.ru news.y...
1    sweetrading.ru sweetrading.ru sweetrading.ru 1...
2    ru.oriflame.com ru.oriflame.com ru.oriflame.co...
3    translate-tattoo.ru nadietah.ru 1obl.ru 1obl.r...
4    mail.rambler.ru news.rambler.ru mail.rambler.r...
Name: user_json, dtype: object

In [10]:
%%time
df['urls'] = df['user_json'].apply(get_domains, 0)

CPU times: user 1min 16s, sys: 330 ms, total: 1min 16s
Wall time: 1min 16s


### Деление на train и test сеты, обучение модели, предсказания для test-сета

Давайте теперь оценим размер нашего train и test сетов. Train set:

In [11]:
len(df[~((df.gender == '-') & (df.age == '-'))])

36138

Test set:

In [12]:
len(df[(df.gender == '-') & (df.age == '-')])

5000

In [13]:
len(df) # Весь датасет

41138

In [14]:
df_train = df[~((df.gender == '-') & (df.age == '-'))]
df_test = df[(df.gender == '-') & (df.age == '-')]

### TFIdf

Воспользуемся известной нам моделью. По умолчанию, встроенный токенайзер разобьет нам домены по точке, поэтому даем модели опцию, которая описывает регулярным выражением, как выделять токены. В нашем случае, токены - все что не пробелы.
Вторая опция позволяет не считать число посещений каждого сайта, а просто выставлять флаг 1, если сайт был посещен. Она сработала лучше, чем дефолтная False.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    token_pattern = '\S+',
    binary=True,
)

Мы пока только объявили модель, но не обучили. 

### Naive Bayes

Наивный баейсовский классификатор хорошо зарекомендовал себя в спам-фильтрах. Мы используем многоклассовую его версию. В качестве добавочного бенефита - он работает гораздо быстрее логистической регрессии.

In [16]:
from sklearn.naive_bayes import MultinomialNB

In [17]:
nb = MultinomialNB()

## Pipeline

Pipeline позволет нам построить сложную модель, состоящую из нескольких шагов-преобразований данных и использовать ее как простую. Разберите примеры из документации этого класса 
http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
а так же мой кернел на Kaggle для каких-то продвинутых трюков
https://www.kaggle.com/metadist/work-like-a-pro-with-pipelines-and-feature-unions

Определяем пайплайн

In [18]:
from sklearn.pipeline import Pipeline

model = Pipeline([
    ('tfdif', tfidf),
    ('nb', nb),
])

Обучаем эту составную модель

In [19]:
model.fit(df_train['urls'],df_train['target'])

Pipeline(memory=None,
     steps=[('tfdif', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...True,
        vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

### Сохранение модели

In [20]:
import pickle
    
with open('/tmp/artemp.pickle', 'wb') as f:
    pickle.dump(model, f)


Для самопроверки вы можете локально оттестировать свой скрипт, используя следующую команду:

In [21]:
#!tail -n1000 gender_age_dataset.txt | python3 project01_gender-age.py > output.json

### Сhecker

Приводится тут для наглядности. Можете сделать copy-paste в `project01_gender-age.py` и заменить строчки для ввода-вывода на те, где используется stdin/stdout

In [22]:
import pandas as pd
import re, json
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote

def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    #if netloc is not None: return str(netloc.encode('utf8')).strip()
    if netloc is not None: return str(netloc.strip())
    return None

def get_domains(j):
    return " ".join([url2domain(d['url']) for d in json.loads(j)['visits'] ])

#считываем данные
data = pd.DataFrame(columns=['gender','age','uid','user_json'])
s=0
import sys
#for line in sys.stdin:
test_lines = !tail -n5000 $file_path
for line in test_lines:
    data.loc[s] = line.strip().split("\t")
    s+=1

In [23]:
# преобразование данных
data['urls'] = data['user_json'].apply(get_domains, 0)

In [24]:
# считывание модели
import pickle

with open('/tmp/artemp.pickle', 'rb') as f:
    model = pickle.load( f)    

# предсказание
pred = model.predict(data['urls'])
data['gender'] = [s[:1] for s in pred]
data['age'] = [s[1:] for s in pred]

# вывод
outf = "out1.json"
#outf = sys.stdout
data[['uid', 'gender', 'age']]\
    .sort_values(by='uid',axis = 0, ascending = True)\
    .to_json(outf, orient='records')

Это решение дает 0.303 на паблике.